In [21]:
import arcpy
import os 

In [6]:
import arcgis
import pdf2image

In [26]:

csv_path = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_cfm\Transposed.csv"
tiff_path = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_cfm\ncld2011.tif"


In [27]:
arcpy.management.MakeXYEventLayer(csv_path, "BeginLong", "BeginLat", "csv")

<Result 'csv'>

In [28]:
# Path to your raster
raster_path = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_cfm\nlcd_2011.tif"



# Calculate statistics
arcpy.management.CalculateStatistics(r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_cfm\nlcd_2011.tif")

# Build attribute table
arcpy.management.BuildRasterAttributeTable(r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_cfm\nlcd_2011.tif", "OVERWRITE")



<Result 'C:\\Users\\coram\\Documents\\geog4057_coramikolajczyk\\Final_Project\\final_cfm\\nlcd_2011.tif'>

In [29]:
output_gdb = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb"
csv_path = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_cfm\Transposed.csv"
xy_layer = "points_layer"
projected_layer = "points_projected"
buffer_output = "points_buffer_289m"

# === 2. Create XY Event Layer from CSV ===
arcpy.management.MakeXYEventLayer(
    table=csv_path,
    in_x_field="BeginLong",  # Replace with your actual column name
    in_y_field="BeginLat",   # Replace with your actual column name
    out_layer=xy_layer,
    spatial_reference=4326  # WGS 1984
)

# === 3. Save to feature class ===
points_fc = f"{output_gdb}\\points"
arcpy.management.CopyFeatures(xy_layer, points_fc)


projected_fc = f"{output_gdb}\\{projected_layer}"
arcpy.management.Project(
    in_dataset=points_fc,
    out_dataset=projected_fc,
    out_coor_system=arcpy.SpatialReference(26960)  
)

# === 5. Create 289-meter buffer ===
buffer_fc = f"{output_gdb}\\{buffer_output}"
arcpy.analysis.Buffer(
    in_features=projected_fc,
    out_feature_class=buffer_fc,
    buffer_distance_or_field="289 Meters"
)

print("✅ Buffer created successfully.")


✅ Buffer created successfully.


In [30]:
# Paths
input_raster = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_cfm\nlcd2011.tif"
buffer_fc = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb"
output_raster = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb\nlcd_clipped"

# Clip the raster using the buffer
arcpy.management.Clip(
    in_raster=input_raster,
    rectangle="#",  # Use the feature geometry instead
    out_raster=output_raster,
    in_template_dataset=buffer_fc,
    nodata_value="0",
    clipping_geometry="ClippingGeometry",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

print("✅ Raster clipped to buffer area.")


<class 'arcgisscripting.ExecuteError'>: Failed to execute. Parameters are not valid.
ERROR 000732: Output Extent: Dataset C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb does not exist or is not supported
Failed to execute (Clip).


In [ ]:
arcpy.env.workspace = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb"
print(arcpy.ListFeatureClasses())

In [ ]:
import arcpy
from arcpy.sa import *

# Enable Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Set environment
arcpy.env.overwriteOutput = True

# Inputs
input_raster = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_cfm\nlcd2011.tif"
mask_fc = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb\points_buffer_289m"
output_masked_raster = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb\nlcd_masked"

# Apply the mask
masked_raster = ExtractByMask(input_raster, mask_fc)
masked_raster.save(output_masked_raster)

print("✅ Raster masked successfully.")


In [ ]:
import arcpy
from arcpy.sa import *

# Enable Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Path to your masked raster
masked_raster_path = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb\nlcd_masked"

# Load the raster
masked_raster = arcpy.Raster(masked_raster_path)

# Get pixel size (assumes square pixels)
pixel_size_x = masked_raster.meanCellWidth
pixel_size_y = masked_raster.meanCellHeight
pixel_area = pixel_size_x * pixel_size_y  # in square meters

# Count valid (non-NoData) pixels
valid_pixels = int(arcpy.GetRasterProperties_management(masked_raster, "COUNT").getOutput(0))

# Calculate total area
total_area_sqm = valid_pixels * pixel_area
total_area_hectares = total_area_sqm / 10000

print(f"✅ Total area: {total_area_sqm:.2f} m² ({total_area_hectares:.2f} hectares)")


In [ ]:
import arcpy

# Path to your raster
raster_path = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb\nlcd_masked"

# Get raster properties
raster = arcpy.Raster(raster_path)

# Print properties
print("Width:", raster.width)
print("Height:", raster.height)
print("Pixel size (X):", raster.meanCellWidth)
print("Pixel size (Y):", raster.meanCellHeight)
print("Spatial reference:", raster.spatialReference.name)
print("Extent:", raster.extent)


In [ ]:
import arcpy

# Path to your masked raster
masked_raster_path = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb\nlcd_masked"

# Load the raster
raster = arcpy.Raster(masked_raster_path)

# Get pixel area (assuming square pixels)
pixel_area = raster.meanCellWidth * raster.meanCellHeight  # in square meters

# Dictionary to store results
class_areas = {}

# Read from the raster attribute table
with arcpy.da.SearchCursor(masked_raster_path, ["Value", "Count"]) as cursor:
    for value, count in cursor:
        area_sqm = count * pixel_area
        area_ha = area_sqm / 10000
        class_areas[value] = {
            "count": count,
            "area_sqm": area_sqm,
            "area_ha": area_ha
        }

# Print results
for value, stats in class_areas.items():
    print(f"Class {value}: {stats['count']} pixels, {stats['area_sqm']:.2f} m² ({stats['area_ha']:.2f} ha)")

print("✅ Area per class calculated.")


In [ ]:
import arcpy
from arcpy.sa import *

# Enable Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Set environment
arcpy.env.overwriteOutput = True

# Paths
project_folder = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project"
masked_raster_path = fr"{project_folder}\final_project.gdb\nlcd_masked"
buffer_fc = fr"{project_folder}\final_project.gdb\points_buffer_289m"
scratch_gdb = fr"{project_folder}\scratch.gdb"

# Create scratch.gdb if it doesn't exist
if not arcpy.Exists(scratch_gdb):
    arcpy.CreateFileGDB_management(project_folder, "scratch.gdb")

# Load the raster
masked_raster = arcpy.Raster(masked_raster_path)
pixel_area = masked_raster.meanCellWidth * masked_raster.meanCellHeight  # m²

# Dictionary to store results
buffer_class_areas = {}

# Loop through each buffer
with arcpy.da.SearchCursor(buffer_fc, ["OID@", "SHAPE@"]) as buffer_cursor:
    for buffer_id, buffer_geom in buffer_cursor:
        # Extract raster within buffer
        extracted = ExtractByMask(masked_raster, buffer_geom)

        # Save to scratch.gdb with unique name
        temp_raster_path = fr"{scratch_gdb}\temp_extract_{buffer_id}"
        extracted.save(temp_raster_path)

        # Build attribute table
        arcpy.management.BuildRasterAttributeTable(temp_raster_path, "OVERWRITE")

        # Read class values and counts
        class_areas = {}
        with arcpy.da.SearchCursor(temp_raster_path, ["Value", "Count"]) as cursor:
            for value, count in cursor:
                area_sqm = count * pixel_area
                area_ha = area_sqm / 10000
                class_areas[value] = {
                    "count": count,
                    "area_sqm": area_sqm,
                    "area_ha": area_ha
                }

        buffer_class_areas[buffer_id] = class_areas

# Print results
for buffer_id, classes in buffer_class_areas.items():
    print(f"Buffer {buffer_id}:")
    for value, stats in classes.items():
        print(f"  Class {value}: {stats['count']} pixels, {stats['area_sqm']:.2f} m² ({stats['area_ha']:.2f} ha)")

print("✅ Area per class within each buffer calculated.")


In [33]:

aprx = arcpy.mp.ArcGISProject("CURRENT")
print("Maps:", [m.name for m in aprx.listMaps()])
print("Layouts:", [l.name for l in aprx.listLayouts()])


Maps: ['Land Coverage of in Miami Florida ']
Layouts: ['Layout', 'layout2', 'Layout1']


In [ ]:
import arcpy

# Set paths
input_fc = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb\points_buffer_289m"
output_folder = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project"
output_shapefile = output_folder + r"\buffer_export.shp"

# Export to shapefile
arcpy.conversion.FeatureClassToShapefile([input_fc], output_folder)

print("✅ Shapefile exported successfully.")


In [31]:
import arcpy

# Input raster
input_raster = r"C:\Users\coram\Documents\ArcGIS\Projects\final_project\final_project.gdb\nlcd_masked"

# Output folder and shapefile name
output_folder = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project"
output_shapefile = output_folder + r"\nlcd_masked_export.shp"

# Convert raster to polygon
arcpy.conversion.RasterToPolygon(
    in_raster=input_raster,
    out_polygon_features=output_shapefile,
    simplify="NO_SIMPLIFY",
    raster_field="Value"
)

print("✅ Raster converted to shapefile successfully:")
print(output_shapefile)


✅ Raster converted to shapefile successfully:
C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\nlcd_masked_export.shp


In [36]:
import arcpy

# Access the current project and layout
aprx = arcpy.mp.ArcGISProject("CURRENT")
layout = aprx.listLayouts("Layout")[0]

# Set output path
output_pdf = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_project_map1.pdf"

# Export layout to PDF
layout.exportToPDF(output_pdf)

print("✅ Layout exported to PDF:")
print(output_pdf)


✅ Layout exported to PDF:
C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_project_map1.pdf


In [35]:
import arcpy

# Access the current project and layout
aprx = arcpy.mp.ArcGISProject("CURRENT")
layout = aprx.listLayouts("Layout1")[0]

# Set output path
output_pdf = r"C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_project_map.pdf"

# Export layout to PDF
layout.exportToPDF(output_pdf)

print("✅ Layout exported to PDF:")
print(output_pdf)


✅ Layout exported to PDF:
C:\Users\coram\Documents\geog4057_coramikolajczyk\Final_Project\final_project_map.pdf


The goal of this project was to calculate the percent impervious cover and compare transects found in both urban and natural settings in and around Miami Florida. The transect data was found from the Florida Fish and Wildlife Conservation Commission and the land cover data was found from the NLCD database and are both free and publicly available. 
To begin the project, the necessary modules are imported along with the CVS and NLCD raster files into the workspace. The NLCD2011 tiff file may need to be converted to a raster file and after that is completed, statistics need to be calculated to be able to run in arcgis. Once the raster is readable and has an attribute table, you can see the data related to the pixels. As for the CSV file, it is then converted to an XY table then points can be created from the BeginLong and BeginLat data. A buffer is then created around the new “transect” points of 289 meters. The buffer size was predetermined by wetland scientists in different studies. The raster and tiff were clipped and masked so that way the NLCD data was only present within the buffer zone. After this, the raster data is converted to square meters for each classification found within the NLCD dataset. These were then added to the map along with the total area. The results were then exported as a shapefile and CSV file along with two layouts presenting the results in a natural setting and an urban setting. Finally, I tried to clean it up as best as possible to be able to run it in the python toolbox. 




In [7]:
from pdf2image import convert_from_path
from IPython.display import display

# Convert PDF to images
images = convert_from_path("C:\\Users\\coram\\Documents\\geog4057_coramikolajczyk\\Final_Project\\final_project_map.pdf")

# Display the first page
display(images[0])


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?